## Setup

### Download data from GEO dataset

1. Navigate to the GEO dataset page:
    https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE240429

2. Download and unzip all files (if necessary) *except* the following:
    - barcodes.tsv.gz
    - features.tsv.gz
    - matrix.mtx.gz

## Preprocess data


In [ ]:
import os

if os.path.basename(os.getcwd()) == 'tutorials':
    # Change to parent directory
    os.chdir('..')

In [ ]:
from src.preprocess.pipeline import TriplexPipeline, get_config                        
from src.preprocess.pipeline.utils import get_available_gpus

In [ ]:
# Example of processing Visium data for training
visium_config = {
    'mode': 'train',
    'platform': 'visium',
    'input_dir': './GSE240429',  # Replace with actual path
    'output_dir': 'input/GSE240429',
    'slide_ext': '.tif',
    'save_neighbors': True
}

pipeline = TriplexPipeline(visium_config)
pipeline.run_pipeline() # This will run preprocessing and feature extraction


## Model Training


```bash
NUM_GPU=2
MODE=cv
DATASET="GSE240429"

# Define models to train
MODELS=("TRIPLEX" "StNet" "EGN" "BLEEP")

# Submit jobs for each model
for MODEL in "${MODELS[@]}"; do
     python src/main.py --config_name $DATASET/$MODEL --gpu $NUM_GPU --mode $MODE
done
```

## Model Evaluation



```bash
declare -A MODELS=(
     ["TRIPLEX"]="Log name for TRIPLEX"
     ["StNet"]="Log name for StNet"
     ["EGN"]="Log name for EGN"
     ["BLEEP"]="Log name for BLEEP"
)

DATASET="GSE240429"
# Loop through each model
for MODEL in "${!MODELS[@]}"; do
     LOG_NAME=${MODELS[$MODEL]}
     python src/main.py --config_name $DATASET/$MODEL --gpu 1 --mode eval --log_name $LOG_NAME
     python src/experiment/agg_results.py --dataset $DATASET --model $MODEL --log_name $LOG_NAME
done
```